Name: Obembe, Olufemi   
Assignment:  Week 9--Best Model Selection and Hyperparameter Tuning   
Date:  February 9, 2022.

**Import the Necessary Libraries for loading the data**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Import the dataset

df = pd.read_csv("/content/drive/MyDrive/DSC 550/Loan_Train.csv")
df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [ ]:
# Check the datatypes

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            614 non-null    object 
 1   Gender             601 non-null    object 
 2   Married            611 non-null    object 
 3   Dependents         599 non-null    object 
 4   Education          614 non-null    object 
 5   Self_Employed      582 non-null    object 
 6   ApplicantIncome    614 non-null    int64  
 7   CoapplicantIncome  614 non-null    float64
 8   LoanAmount         592 non-null    float64
 9   Loan_Amount_Term   600 non-null    float64
 10  Credit_History     564 non-null    float64
 11  Property_Area      614 non-null    object 
 12  Loan_Status        614 non-null    object 
dtypes: float64(4), int64(1), object(8)
memory usage: 62.5+ KB


The number of dependents seem to have been imported as object instead of integer.  Aside from that, all other variables were imported propertly.

In [ ]:
# Check the values in Depedents

df['Dependents'].value_counts()

0     345
1     102
2     101
3+     51
Name: Dependents, dtype: int64

Dependents was actually an object, we can remap the values to eliminate the 3+ among the values.

In [ ]:
dep_map = {'0':'None','1':'One','2':'Two','3+':'Above_two'}
df['No_Dependents'] = df['Dependents'].map(dep_map)

In [ ]:
df['No_Dependents'].value_counts()

None         345
One          102
Two          101
Above_two     51
Name: No_Dependents, dtype: int64

In [ ]:
# We can delete Dependents and rename No_dependents as Dependents

df.drop('Dependents',axis=1,inplace=True)

**Drop the column "Load_ID**

In [ ]:
df.drop('Loan_ID',axis=1,inplace=True)

**Check for missing data and drop rows with missing data**

In [ ]:
df.isnull().sum()

Gender               13
Married               3
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
No_Dependents        15
dtype: int64

In [ ]:
# check number of columns and rows

df.shape

(614, 12)

In [ ]:
# Drop rows with missing data

df = df.dropna()

In [ ]:
df.shape

(480, 12)

We lost 134 rows of data

**Convert the categorical features into dummy variables**

In [ ]:
df = pd.get_dummies(df,drop_first=True)

In [ ]:
df.shape

(480, 15)

In [ ]:
df.head()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Gender_Male,Married_Yes,Education_Not Graduate,Self_Employed_Yes,Property_Area_Semiurban,Property_Area_Urban,Loan_Status_Y,No_Dependents_None,No_Dependents_One,No_Dependents_Two
1,4583,1508.0,128.0,360.0,1.0,1,1,0,0,0,0,0,0,1,0
2,3000,0.0,66.0,360.0,1.0,1,1,0,1,0,1,1,1,0,0
3,2583,2358.0,120.0,360.0,1.0,1,1,1,0,0,1,1,1,0,0
4,6000,0.0,141.0,360.0,1.0,1,0,0,0,0,1,1,1,0,0
5,5417,4196.0,267.0,360.0,1.0,1,1,0,1,0,1,1,0,0,1


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 480 entries, 1 to 613
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   ApplicantIncome          480 non-null    int64  
 1   CoapplicantIncome        480 non-null    float64
 2   LoanAmount               480 non-null    float64
 3   Loan_Amount_Term         480 non-null    float64
 4   Credit_History           480 non-null    float64
 5   Gender_Male              480 non-null    uint8  
 6   Married_Yes              480 non-null    uint8  
 7   Education_Not Graduate   480 non-null    uint8  
 8   Self_Employed_Yes        480 non-null    uint8  
 9   Property_Area_Semiurban  480 non-null    uint8  
 10  Property_Area_Urban      480 non-null    uint8  
 11  Loan_Status_Y            480 non-null    uint8  
 12  No_Dependents_None       480 non-null    uint8  
 13  No_Dependents_One        480 non-null    uint8  
 14  No_Dependents_Two        4

**Split the data into a training and test set, where the Loan_ID is the target**

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# Prepare the features into X and y variables

X = df.drop('Loan_Status_Y',axis=1)
y = df['Loan_Status_Y']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

**Create a Pipeline with a min-max scaler and KNN classifier**

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

In [ ]:
# Instantiating the parameters

scaler = MinMaxScaler()
knn = KNeighborsClassifier(n_neighbors=5,n_jobs=-1)
pipe = Pipeline([("scaler",scaler),("knn",knn)])

In [ ]:
# Fit the model

kmodel = pipe.fit(X_train,y_train)

In [ ]:
kmodel

Pipeline(steps=[('scaler', MinMaxScaler()),
                ('knn', KNeighborsClassifier(n_jobs=-1))])

In [ ]:
y_pred = kmodel.predict(X_test)

*Evaluate the Model Accuracy*

In [ ]:
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report,plot_confusion_matrix

In [ ]:
accuracy_score(y_test,y_pred)

0.7708333333333334

**Create a Search Space for the model where n_neighbors ranges from 1:10**

In [ ]:
search_space = [{"knn__n_neighbors":[1,2,3,4,5,6,7,8,9,10]}]

**Fitting the model with GridSearchCV**

In [ ]:
kmodel2 = GridSearchCV(pipe,search_space,cv=5,verbose=0).fit(X_train,y_train)

In [ ]:
kmodel2

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', MinMaxScaler()),
                                       ('knn',
                                        KNeighborsClassifier(n_jobs=-1))]),
             param_grid=[{'knn__n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]}])

In [ ]:
kmodel2.best_estimator_

Pipeline(steps=[('scaler', MinMaxScaler()),
                ('knn', KNeighborsClassifier(n_jobs=-1, n_neighbors=10))])

In [ ]:
y_pred2 = kmodel2.predict(X_test)

In [ ]:
accuracy_score(y_test,y_pred2)

0.7604166666666666

**Expanding the search space to  include Logistic Regression and Random Forest**

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [52]:
# Create a pipeline

pipe2 = Pipeline([("scaler",MinMaxScaler()),
                  ("classifier",RandomForestClassifier())])


In [55]:
search_space2 = [{"classifier":[LogisticRegression()],
                  "classifier__penalty":['l1','l2'],
                  "classifier__C":np.logspace(0,4,10)},
                 {"classifier":[KNeighborsClassifier()],
                  "classifier__n_neighbors":[1,2,3,4,5,6,7,8,9,10],
                  "classifier__weights":['uniform','distance']},
                 {"classifier":[RandomForestClassifier()],
                  "classifier__n_estimators":[10,100,1000],
                  "classifier__max_features": [1,2,3],
                  "classifier__max_depth":[5,10,None]}]

In [56]:
Clf = GridSearchCV(pipe2,search_space2,cv=5,verbose=0)
Clf.fit(X_train,y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
50 fits failed out of a total of 335.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
50 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/pipeline.py", line 394, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py", line 1461, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  Fi

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', MinMaxScaler()),
                                       ('classifier',
                                        RandomForestClassifier())]),
             param_grid=[{'classifier': [LogisticRegression(C=2.7825594022071245)],
                          'classifier__C': array([1.00000000e+00, 2.78255940e+00, 7.74263683e+00, 2.15443469e+01,
       5.99484250e+01, 1.66810054e+02, 4.64158883e+02, 1.29154967e+03,
       3.59381366e+03, 1.00000000e+04]),
                          'classifier__penalty': ['l1', 'l2']},
                         {'classifier': [KNeighborsClassifier()],
                          'classifier__n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9,
                                                      10],
                          'classifier__weights': ['uniform', 'distance']},
                         {'classifier': [RandomForestClassifier()],
                          'classifier__max_depth': [5, 10, None],
  

**Best Model and Hyperparameters Found**

In [57]:
Clf.best_estimator_

Pipeline(steps=[('scaler', MinMaxScaler()),
                ('classifier', LogisticRegression(C=2.7825594022071245))])

In [59]:
# Instantiating the parameters

scaler = MinMaxScaler()
Log_pipe = Pipeline([("scaler",scaler),("classifier",LogisticRegression(C=2.7826))])

In [61]:
# Fit the model

BestModel = Log_pipe.fit(X_train,y_train)

In [62]:
y_pred3 = BestModel.predict(X_test)

In [63]:
accuracy_score(y_test,y_pred3)

0.8229166666666666

## Summary of Results

Using the K-Nearest Neighbor with n=5 produces accuracy score of 77.08%, when we employed a gridsearch to search for the optimal number of n, 10 was picked but eventually produced a poorer results (76.04%) than when n = 5.

When we expanded the search space to include 2 other algorithms:  RandomForest and LogisticRegression, and allowing GridSearch to perform the hyperparameter tuning, Logistic regression model was returned as the best model with C = 2.7826.  

Fitting logistic regression model on the dataset produced a higher accuracy score of 82.3%